# Deuxième partie : Détection des tumeurs cérébrales avec YOLOv8

Cette partie consiste à classer les images de tumeurs cérébrales et à localiser les tumeurs à l'aide du modèle YOLOv8.

In [ ]:
# Imports et configuration initiale


## 1. Afficher un échantillon d'images pour chaque classe avec les boîtes englobantes

## 3. Filtrer et copier les images et labels

Pour chaque image :
- Vérifier si un fichier .txt correspondant existe dans le répertoire des labels
- Si un label est trouvé : copier l'image et le label vers le dossier approprié (train/valid/test)
- Si aucun label n'est trouvé : afficher un avertissement et sauter l'image

## 4. Créer le fichier data.yaml sans augmentations

Ce fichier contient :
- Les chemins des dossiers d'entraînement, de validation et de test
- Le nombre de classes et le nom de chaque classe
- Configuration sans augmentations de données

## 5. Créer le fichier data2.yaml avec augmentations

Ce fichier contient la même structure que data.yaml mais avec des augmentations de données activées

## 6. Compter le nombre d'images et d'étiquettes

Compter les images et labels présents dans les ensembles d'entraînement et de validation

In [ ]:
import os

train_images = '../Data/Raw/Data_Brain/'
train_labels = '../Data/Raw/Data_Brain/'
valid_images = '../Data/Raw/Data_Brain/'
valid_labels = '../Data/Raw/Data_Brain/'

train_img_count = len([f for f in os.listdir(train_images) if f.endswith(('.jpg', '.png', '.jpeg'))])
train_label_count = len([f for f in os.listdir(train_labels) if f.endswith('.txt')])
valid_img_count = len([f for f in os.listdir(valid_images) if f.endswith(('.jpg', '.png', '.jpeg'))])
valid_label_count = len([f for f in os.listdir(valid_labels) if f.endswith('.txt')])

print(f"tain:")
print(f"  - Images: {train_img_count}")
print(f"  - Labels: {train_label_count}")
print(f"\nvalidation")
print(f"  - Images: {valid_img_count}")
print(f"  - Labels: {valid_label_count}")

## 7. Vérifier la correspondance images-labels et nettoyer les données

- Vérifier que chaque image possède un label correspondant
- Supprimer les images sans label
- Supprimer les labels sans image correspondante

In [ ]:
import os

datasets = {
    'train': {
        'images': 'filtered_data/train/images/',
        'labels': 'filtered_data/train/labels/'
    },
    'valid': {
        'images': 'filtered_data/valid/images/',
        'labels': 'filtered_data/valid/labels/'
    },
    'test': {
        'images': 'filtered_data/test/images/',
        'labels': 'filtered_data/test/labels/'
    }
}

def verify_and_clean(images_dir, labels_dir):
    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        print(f"Les répertoires {images_dir} ou {labels_dir} n'existent pas.")
        return
    
    images = {os.path.splitext(f)[0]: f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))}
    labels = {os.path.splitext(f)[0]: f for f in os.listdir(labels_dir) if f.endswith('.txt')}
    
    images_removed = 0
    labels_removed = 0
    
    for img_name in list(images.keys()):
        if img_name not in labels:
            os.remove(os.path.join(images_dir, images[img_name]))
            images_removed += 1
            print(f"Image supprimée (pas de label): {images[img_name]}")
    
    for label_name in list(labels.keys()):
        if label_name not in images:
            os.remove(os.path.join(labels_dir, labels[label_name]))
            labels_removed += 1
            print(f"Label supprimé (pas d'image): {labels[label_name]}")
    
    remaining_count = len(images) - images_removed
    print(f"\nRésumé: {images_removed} images et {labels_removed} labels supprimés")
    print(f"Paires valides restantes: {remaining_count}\n")

for dataset_name, paths in datasets.items():
    print(f"=== Vérification de {dataset_name} ===")
    verify_and_clean(paths['images'], paths['labels'])

## 8. Entraîner le modèle YOLOv8

Lancer l'entraînement du modèle YOLO en définissant les hyperparamètres appropriés

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt') 

model.train(
    data='data.yaml',      
    epochs=100,             
    batch=-1,
    cos_lr = True,
    patience = 10,                       
    device=0,    
    save=True              
)


## 9. Évaluer et tester le modèle

Évaluer et tester le modèle après l'entraînement pour mesurer :
- Les performances
- La précision
- La capacité à généraliser sur des données inédites

## 10. Sauvegarder le modèle entraîné

Sauvegarder le modèle entraîné pour une utilisation future